In [ ]:
# permutations of actions and queue

In [ ]:
import pandas as pd
import numpy as np
import random
from random import randrange
import time
import os
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
import itertools
from itertools import permutations
from itertools import combinations

from copy import deepcopy

In [ ]:
class world:
    def __init__(self,Size):
        self.Size = Size
        self.Fig = None
        self.Grid = None
        self.Coin_Locations = []

    def Generate_Map(world):
        # generate a grid with the dimensions of world.Size
        grid = np.zeros([world.Size,world.Size])
    
        fig = plt.figure(figsize = [5,5])
        plt.xlim(0,world.Size)
        plt.ylim(0,world.Size)

        world.Grid = grid
        world.Fig = fig

In [ ]:
class entity:
    def __init__(self,world):
        self.world = world
        self.IsSpawned = False

        if self.IsSpawned == False:
            self.Location = [0,0]
            
        elif self.IsSpawned == True:
            x_loc = world.Grid[np.where(world.Grid[:,0] == max(world.Grid))]
            print('x: ',x_loc)
            y_loc = world.Grid[np.where(world.Grid[0,:] == max(world.Grid))]
            print('y: ',y_loc)
            self.Location = [x_loc,y_loc]

        self.Speed = 6
        self.Coins = 0


In [ ]:
stage = world(10)
stage.Generate_Map()
actor = entity(stage)

In [ ]:
queue = {
    'action': [],
    'destination': [],
    'metadata': []
}


In [ ]:
# create observer class
class Observer:
    def __init__(self, name):
        self.name = name

    def update(self, queue):
        print(f'{self.name} received: {queue}')


class Observable:
    def __init__(self):
        self.observers = []

    def add_observer(self, observer):
        self.observers.append(observer)

    def remove_observer(self, observer):
        self.observers.remove(observer)

    def notify_observers(self, queue):
        for observer in self.observers:
            observer.update(queue)

    def update_queue(self, queue):
        self.notify_observers(queue)
    

In [ ]:
def adjacent_locations(pos):
    adjacent_locations_list = []
    # for an 8 directional grid
    for i in range(-1,2):
        for j in range(-1,2):
            adjacent_locations_list.append([pos[0]+i,pos[1]+j])
    adjacent_locations_list.remove(pos)
    return adjacent_locations_list

# Chebyshev 
def chebyshev_distance(pos1,pos2):
    return max(abs(pos1[0]-pos2[0]),abs(pos1[1]-pos2[1]))

In [ ]:
subaction_dict = {
    '0': 'Move',                # needs a target
    '1': 'Object (Bonus)',      # needs a target
    '2': 'Attack',              # needs a target
    '3': 'Dash',
    '4': 'Disengage',
    '5': 'Dodge',
    '6': 'Object',              # needs a target
}

In [ ]:
##### Fleshing Out the Move Action #####
# the Move Action moves them 1 Space to a new Space
# the new Space is the target
# and the target choices the entity has to pick from  

# because of the problem of any Move action after the first not knowing where the entity is, which of the two solutions is more efficient:
# 1. Rendering: every time a move action is in the permutation list, render the world and the entity's location
# 2. Location Incorporation: when randomly generating the permutations, the locations within 6 spaces of the entity are included in the permutation list, the validity rules are updated accordingly

def movement_range(entity):
    entity_pos = entity.Location
    movement_range = []
    for x in range(-entity.Speed,entity.Speed+1):
        for y in range(-entity.Speed,entity.Speed+1):
            if abs(x) + abs(y) <= entity.Speed:
                movement_range.append([entity_pos[0]+x,entity_pos[1]+y])
    #print('Spaces within Movement Range: ',len(movement_range))
    return movement_range

# what if there was a function called flesh_out_move_perms that went through the list of initial permutations (that only had 0-6) and where ever there was a 0, 
# it would create a new permutation in which a target was chosen from the movement_range of the entity
# then a second function to check the validity of movement

def validate_movement(perms,entity):

    for perm in perms:
        #print(perm)
        if 0 in perm:
            # moves_taken should be a list of all the items in prem that are a list with 0 as the first element
            moves_taken = [item[1] for item in perm if type(item) == list and item[0] == 0]

            # check that the distance from one item to the next is one, if not, remove the permutation
            for i in range(len(moves_taken)-1):
                # if the first space is not 1 space or less from the entity's location, remove the permutation
                if chebyshev_distance(moves_taken[i],entity.Location) > 1:
                    perms.remove(perm)
                    break
                
                # use chevyshev distance for this
                if chebyshev_distance(moves_taken[i],moves_taken[i+1]) > 1:
                    perms.remove(perm)
                    break


In [ ]:
# What Should Be Happening?


# this example permutation is an action-series representing two move actions
# these permutations can range from 1 to 8 numbers in length, and contain numbers 0 through 6. 
# I'm only trying to flesh out the permutations that contain 0s, and the 0s represent move actions
# Example Permutation: [0,0]

# Step 1
# Create 8 new ones
# [ [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0] ]

# Step 2
# Replace the first 0 in each with an adjacent location
# [ [[0,[-1,-1]],0], [[0,[-1,1]],0], [[0,[1,-1]],0], [[0,[1,0]],0], [[0,[0,1]],0], [[0,[-1,0]],0], [[0,[0,-1]],0], [[0,[1,1]],0] ]

# Step 3
# Create 8 copies of each of the above
# [ [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0]]
# [ [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0]]
# [ [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0]]
# [ [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0]]
# [ [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0]]
# [ [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0]]
# [ [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0]]
# [ [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0]]

# Step 4
# Replace the second 0 in each with an adjacent location to the first 0
# [ [ [0,[-1,-1]],[0,[-2,-2]] ], [ [0,[-1,-1]],[0,[-1,-2]] ], [ [0,[-1,-1]],[0,[0,-2]] ], [ [0,[-1,-1]],[0,[0,-1]] ], [ [0,[-1,-1]],[0,[0,0]] ], [ [0,[-1,-1]],[0,[-1,0]] ], [ [0,[-1,-1]],[0,[-2,0]] ], [ [0,[-1,-1]],[0,[-2,-1]] ] ]
# [ [ [0,[-1,1]],[0,[-2,-1]] ],  [ [0,[-1,1]],[0,[-1,-1]] ],  [ [0,[-1,1]],[0,[0,-1]] ],  [ [0,[-1,1]],[0,[0,0]] ],   [ [0,[-1,1]],[0,[0,1]] ],  [ [0,[-1,1]],[0,[-1,1]] ],  [ [0,[-1,1]],[0,[-2,1]] ],  [ [0,[-1,1]],[0,[-2,0]] ] ]
# [ [ [0,[1,-1]],[0,[0,-2]] ],   [ [0,[1,-1]],[0,[1,-2]] ],   [ [0,[1,-1]],[0,[1,0]] ],   [ [0,[1,-1]],[0,[0,0]] ],   [ [0,[1,-1]],[0,[2,0]] ],  [ [0,[1,-1]],[0,[2,-1]] ],  [ [0,[1,-1]],[0,[2,-2]] ],  [ [0,[1,-1]],[0,[0,-2]] ] ]
# [ [ [0,[1,0]],[0,[2,-1]] ],    [ [0,[1,0]],[0,[1,-1]] ],    [ [0,[1,0]],[0,[0,-1]] ],   [ [0,[1,0]],[0,[0,0]] ],    [ [0,[1,0]],[0,[0,1]] ],   [ [0,[1,0]],[0,[1,1]] ],    [ [0,[1,0]],[0,[2,1]] ],    [ [0,[1,0]],[0,[2,0]] ] ]
# [ [ [0,[0,1]],[0,[-1,0]] ],    [ [0,[0,1]],[0,[0,0]] ],     [ [0,[0,1]],[0,[1,0]] ],    [ [0,[0,1]],[0,[1,1]] ],    [ [0,[0,1]],[0,[1,2]] ],   [ [0,[0,1]],[0,[0,2]] ],    [ [0,[0,1]],[0,[-1,2]] ],   [ [0,[0,1]],[0,[-1,1]] ] ]
# [ [ [0,[-1,0]],[0,[-2,1]] ],   [ [0,[-1,0]],[0,[-1,1]] ],   [ [0,[-1,0]],[0,[0,1]] ],   [ [0,[-1,0]],[0,[0,0]] ],   [ [0,[-1,0]],[0,[0,-1]] ], [ [0,[-1,0]],[0,[-1,-1]] ], [ [0,[-1,0]],[0,[-2,-1]] ], [ [0,[-1,0]],[0,[-2,0]] ] ]
# [ [ [0,[0,-1]],[0,[1,-2]] ],   [ [0,[0,-1]],[0,[0,-2]] ],   [ [0,[0,-1]],[0,[-1,-2]] ], [ [0,[0,-1]],[0,[-1,-1]] ], [ [0,[0,-1]],[0,[-1,0]] ], [ [0,[0,-1]],[0,[0,0]] ],   [ [0,[0,-1]],[0,[1,0]] ],   [ [0,[0,-1]],[0,[1,-1]] ] ]
# [ [ [0,[1,1]],[0,[0,0]] ],     [ [0,[1,1]],[0,[1,0]] ],     [ [0,[1,1]],[0,[2,0]] ],    [ [0,[1,1]],[0,[2,1]] ],    [ [0,[1,1]],[0,[2,2]] ],   [ [0,[1,1]],[0,[1,2]] ],    [ [0,[1,1]],[0,[0,2]] ],    [ [0,[1,1]],[0,[0,1]] ] ]

# Step 5
# put the indidual action-series into a single list


# what is the ideal output?
# the ideal output would come out as a single list such as the following:
# [
#     [ [0,[-1,-1]],[0,[-2,-2]] ], 
#     [ [0,[-1,-1]],[0,[-1,-2]] ], 
#     [ [0,[-1,-1]],[0,[0,-2]] ], 
#     [ [0,[-1,-1]],[0,[0,-1]] ], 
#     [ [0,[-1,-1]],[0,[0,0]] ], 
#     [ [0,[-1,-1]],[0,[-1,0]] ], 
#     [ [0,[-1,-1]],[0,[-2,0]] ], 
#     [ [0,[-1,-1]],[0,[-2,-1]] ]
#
#
# ]

In [ ]:
# use a recursive approach to create the process_move_actions function
def generate_permutations(action_series_list, entity, depth=0):
    
    # while this function goes through individual action_series, the function above it (flesh_out_move) goes through a list of action_series
    # action_series_list is supposed to be a single action_series, not a compilation of valid_actions


    if type(action_series_list) == int:
        action_series_list = [action_series_list]
    else:
        action_series_list = list(action_series_list)
    
    print(f"Action series list: {action_series_list}")

    original_list = deepcopy(action_series_list)
    print(f"Current depth: {depth}, Current actions: {action_series_list}")
    max_depth = len(action_series_list)
    full_list = []

    if 0 not in action_series_list:
        print(f"(End 1) Reached end of list. Returning: {action_series_list}")
        print('')
        full_list.append(action_series_list)
        print(f"Full list: {full_list}")
        return [action_series_list]

    if depth == 0:
        adjacent_locations_list = adjacent_locations(entity.Location)
        #print(f"Adjacent locations (Entity): {adjacent_locations_list}")

    else:
        # what needs to happen is if it's the first 0 in the list, it needs to check the entity's location
        # first find out if there is another 0
        number_of_zeros = original_list.count(0)
        # then find the number of zeroes before the current depth, they won't be 0s anymore so instead check for lists
        previous_zeros = [item for item in action_series_list if type(item) == list]

        if number_of_zeros == 1:
            adjacent_locations_list = adjacent_locations(entity.Location)
            #print(f"Adjacent locations (Entity): {adjacent_locations_list}")

        elif number_of_zeros > 1:
            if len(previous_zeros) == 0:
                adjacent_locations_list = adjacent_locations(entity.Location)
                #print(f"Adjacent locations (Entity): {adjacent_locations_list}")
            else:
                adjacent_locations_list = adjacent_locations(previous_zeros[-1][1])


    if depth == max_depth:
        print(f"(End 2) Reached max depth. Returning: {[action_series_list]}")
        print('')
        full_list.append([action_series_list])
        return [action_series_list]
    
    permutations = []
    for move in adjacent_locations_list: #[[-1,-1], [-1,1], [1,-1], [1,0], [0,1], [-1,0], [0,-1], [1,1]]:

        if action_series_list[depth] == 0:
            #print(f"last move: ",min([abs(depth-1),0]))
            last_move = action_series_list[min([abs(depth-1),0])]
            #print(f"First zero location: {action_series_list[depth]}, Last move: {last_move}")

            new_actions = action_series_list.copy()
            
            print(f"Current new actions: {new_actions}")
            
            new_actions[depth] = [0, move]

            print(f"Adding move {move} at depth {depth}")
            full_list.append([new_actions])
            permutations.extend(generate_permutations(new_actions, entity, depth + 1))
            print('Printing Full List at Add: ', full_list)
            
        else: 
            print(f"Skipping move {move} at depth {depth}")
            full_list.append(action_series_list)
            permutations.extend(generate_permutations(action_series_list, entity, depth + 1))
            print('Printing Full List at Skip: ', full_list)

    
    print(f"Returning permutations at depth {depth}: {permutations}")
    return permutations

def flesh_out_move(perms, entity):
    fleshed_out_list = []
    perm_result_lengths = []

    num_of_action_series = len(perms)

    print('Perms: ',perms)


    original_results = []
    for action_series in perms:
        result = generate_permutations(action_series, entity)
        result = list(map(tuple, result))
        

        # I want the final results to be a bit more organized
        action_series_result_length = 0
        for i in result:
            if i not in original_results:
                original_results.extend([i])
                action_series_result_length += 1
        perm_result_lengths.append(action_series_result_length)
        # this counter is to keep track of the number of permutations that have been fleshed out per action series so that the final list can be organized


    #print('Original Results: ',original_results)

    #for i in original_results:
        #if i not in fleshed_out_list:
            #print(f"Point 3a; Adding {i} to fleshed out list")
            #fleshed_out_list.append(original_results)

    #print('Fleshed Out List: ',fleshed_out_list)

    # printing out original results in a more organized way
    for i in range(len(perms)):
        print(f"Original Action Series: {perms[i]}")

        print('Permutation Result Lengths: ',perm_result_lengths[i])

        action_series_first_result = sum(perm_result_lengths[:i])
        action_series_last_result = sum(perm_result_lengths[:i+1])

        for j in range(action_series_first_result,action_series_last_result):
            print(original_results[j])


    #for i in range(num_of_action_series):
    #    print(f"Action Series {i}: {original_results[:sum(perm_result_lengths[:i+1])]}")

    #    for j in range(len(original_results[:sum(perm_result_lengths[:i+1])])):
    #        print(original_results[j])

    print('Point 4; Final Return: ',fleshed_out_list)
    return fleshed_out_list

flesh_out_move_example_list = [[0,0], [0,0,0]]
result = flesh_out_move(flesh_out_move_example_list, actor)


In [ ]:
# what is the ideal output 

In [ ]:
def generate_turns():
    list_of_turns = []
    for length in range(1, 9):  # Turns can be 1 to 8 actions long: 6 Moves, 1 Object Interaction, 1 Action
        for turn in itertools.product(range(6), repeat=length):
            if validate_turn(turn):
                yield turn
                
    return list_of_turns

def validate_turn(turn):
    move_count = sum(1 for action in turn if action == 0)
    object_interact_count = sum(1 for action in turn if action in [1, 3])
    action_count = sum(1 for action in turn if action in [2, 3, 4, 5, 6])
        
    return (move_count <= 6 and 
            object_interact_count <= (2 if 6 in turn else 1) and 
            action_count <= 1)


def calc_rewards(turn, actor):
    # what should be rewarded?
    # conciseness of the action series

    turn_conciseness = 0

    turn_evaluation = 0

    reward = 0
    turn_evaluation += reward
    return turn_evaluation


In [ ]:
def remove_redundant_series(perms):
    pass

# what would be considered redundant?
# - different movement patterns through terrain with no difference in properties

In [ ]:
def Choose_Turn(entity, show_turns: bool):
    turn_options = generate_turns()
    valid_turns = [turn for turn in turn_options if validate_turn(turn)]
    print('Before Fleshing Out:',len(valid_turns))
    # only do the first 100
    fleshed_out_movement_turns = []
    for turn in valid_turns:
        #print(turn)
        new_turn = flesh_out_move(turn,entity)
        print(new_turn)
        fleshed_out_movement_turns.extend(new_turn)


    if show_turns:
        print('Valid Turns:', len(fleshed_out_movement_turns))
        for turn in fleshed_out_movement_turns:
            print(turn)
    
    best_turn = None
    best_score = float('-inf')

    for turn in fleshed_out_movement_turns:
        score = calc_rewards(turn, entity)
        if score > best_score:
            best_turn = turn
            best_score = score
            print(best_score)

    best_turn = list(best_turn)
    return best_turn



In [ ]:
Choose_Turn(actor, False)